<a href="https://www.kaggle.com/code/akashrayhan/qa-similarity-bert?scriptVersionId=111313517" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datacsv/FAQs_test.csv
/kaggle/input/datacsv/FAQs.csv
/kaggle/input/bertforquestionanswering/config.json
/kaggle/input/bertforquestionanswering/tokenizer_config.json
/kaggle/input/bertforquestionanswering/pytorch_model.bin
/kaggle/input/bertforquestionanswering/special_tokens_map.json
/kaggle/input/bertforquestionanswering/vocab.txt


In [2]:
data = pd.read_csv('/kaggle/input/datacsv/FAQs.csv')
test_data = pd.read_csv('/kaggle/input/datacsv/FAQs_test.csv')
data.head(5)

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.


In [3]:
answer_text = ' '.join(data['Answer'].values.astype(str).flatten())
question = test_data['Question']

In [4]:
import torch
from transformers import BertForQuestionAnswering


model = BertForQuestionAnswering.from_pretrained(r'/kaggle/input/bertforquestionanswering/')


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(r'/kaggle/input/bertforquestionanswering/')

In [5]:
def answering_questions(question, answer_text):
    #tokenize

    encoded_dict = tokenizer.encode_plus(text=question,text_pair=answer_text, add_special=True)


    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = encoded_dict['input_ids']

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # Segment Ids
    segment_ids = encoded_dict['token_type_ids']

    # evaluate
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(output['start_logits'])
    answer_end = torch.argmax(output['end_logits'])

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    return answer

In [6]:
answer = []
for i in range(len(test_data['Question'])):
    answer.append(answering_questions(question=test_data['Question'][i], answer_text=answer_text))
test_data['Answer'] = answer

Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Query has 323 tokens.



Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Query has 320 tokens.



Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Query has 320 tokens.



Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Query has 325 tokens.



Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Query has 322 tokens.



In [7]:
test_data.head(5)

,Question,Answer
0,What is the date of his death?,18 april 1955
1,Did Einstein have siblings?,maja
2,Who was his wife?,elsa lowenthal
3,What was Einstein's father's name?,hermann einstein
4,At what institutions did he study?,swiss federal institute of technology in zuric...
